# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo
import datetime

In [2]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['youtube']

#On supprime la collection pour éviter une erreur lorsque qu'on relance le Notebook
collection.delete_many({})

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(5)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


## Question 0

### Netoyer les données

On change le type des valeurs contenues dans les colonnes, ici seulement les colonnes contenant des nombre sont changées en type "numeric"

In [4]:
df_youtube[["category_id", "views", "likes", "dislikes", "comment_total"]] = df_youtube[["category_id", "views", "likes", "dislikes", "comment_total"]].apply(pd.to_numeric)

On découpe les valeurs de la colonne tags en une liste

In [5]:
for i in range (len(df_youtube['tags'])):
    if df_youtube['tags'][i] == '[none]':
        df_youtube['tags'][i] = []
    else:
        df_youtube['tags'][i] = df_youtube['tags'][i].split('|')

/tmp/ipykernel_12/977269364.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youtube['tags'][i] = df_youtube['tags'][i].split('|')
/tmp/ipykernel_12/977269364.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youtube['tags'][i] = []


In [6]:
df_youtube.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,"[logan paul vlog, logan paul, logan, paul, oly...",4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,"[Apple, iPhone 10, iPhone Ten, iPhone, Portrai...",7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,"[apple iphone x hands on, Apple iPhone X, iPho...",2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,"[jacksfilms, parody, parodies, iphone, iphone ...",1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


### Importer les données

In [7]:
DOCUMENTS = df_youtube.to_dict(orient = 'records')

In [8]:
collection.find_one()

In [9]:
collection.insert_many(DOCUMENTS)

## Question 1  

In [10]:
cursor_1 = collection.find({"channel_title":"Apple"})
list(cursor_1)

[{'_id': ObjectId('61d2e78aa4efa5ec84ef27a1'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': ['Apple',
   'iPhone 10',
   'iPhone Ten',
   'iPhone',
   'Portrait Lighting',
   'A11 Bionic',
   'augmented reality',
   'emoji',
   'animoji',
   'Face ID',
   'Apple Pay',
   'camera',
   'smartphone'],
  'views': 7860119,
  'likes': 185853,
  'dislikes': 26679,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': 13.09},
 {'_id': ObjectId('61d2e78aa4efa5ec84ef286b'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': ['Apple',
   'iPhone 10',
   'iPhone Ten',
   'iPhone',
   'Portrait Lighting',
   'A11 Bionic',
   'augmented reality',
   'emoji',
   'animoji',
   'Face ID',
   'Apple Pay',
   'camera',
   'smartphone'],
  'views': 12200526,
  'likes': 2588

## Question 2

In [11]:
cursor_2 = collection.distinct('category_id')
print("Il y a", len(cursor_2),"catégories différentes.")

Il y a 16 catégories différentes.


## Question 3

Les valeurs contenues dans la colonne 'tags' de la DataFrame ont déjà été transformées en liste au niveau de la question 0

## Question 4

In [12]:
cursor_4 = collection.find().sort([("views", -1)])
list(cursor_4)

[{'_id': ObjectId('61d2e78aa4efa5ec84ef2f13'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 22.09},
 {'_id': ObjectId('61d2e78aa4efa5ec84ef2e3e'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
  'views': 38013692,
  'likes': 1950475,
  'dislikes': 75294,
  'comment_total': 713293,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 21.09},
 {'_id': ObjectId('61d2e78aa4efa5ec84ef2851'),
  'video_id': 'tt2k8PGm-TI',
  'title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'channel_title': 'ZaynVEVO',
  'category_id': 10,
  'tags'

## Question 5

In [13]:
cursor_5 = collection.aggregate([{"$group" : {"_id" : "$category_id", "AverageNumberOfViews" : {"$avg" : "$views"}}}])
list(cursor_5)

[{'_id': 19, 'AverageNumberOfViews': 464041.0833333333},
 {'_id': 24, 'AverageNumberOfViews': 1154868.4397251718},
 {'_id': 1, 'AverageNumberOfViews': 1039472.6560846561},
 {'_id': 28, 'AverageNumberOfViews': 924730.548828125},
 {'_id': 20, 'AverageNumberOfViews': 681081.8170731707},
 {'_id': 43, 'AverageNumberOfViews': 8492.5},
 {'_id': 27, 'AverageNumberOfViews': 547582.4700598803},
 {'_id': 22, 'AverageNumberOfViews': 971532.7573696146},
 {'_id': 2, 'AverageNumberOfViews': 607693.8706896552},
 {'_id': 26, 'AverageNumberOfViews': 537665.1806674339},
 {'_id': 10, 'AverageNumberOfViews': 1176553.612},
 {'_id': 25, 'AverageNumberOfViews': 540955.5569823436},
 {'_id': 29, 'AverageNumberOfViews': 1110334.2142857143},
 {'_id': 15, 'AverageNumberOfViews': 651404.3879310344},
 {'_id': 23, 'AverageNumberOfViews': 1240073.294039735},
 {'_id': 17, 'AverageNumberOfViews': 728434.3243902439}]

## Question 6 

In [14]:
cursor_6 = collection.aggregate([{"$group" : {"_id" : "$channel_title", "AverageNumberOfLikes" : {"$avg" : "$likes"}}}, { "$sort" : { "AverageNumberOfLikes" : -1} }])
list(cursor_6)

[{'_id': 'ZaynVEVO', 'AverageNumberOfLikes': 1431683.0},
 {'_id': 'ibighit', 'AverageNumberOfLikes': 1371766.25},
 {'_id': 'melanie martinez', 'AverageNumberOfLikes': 911871.5},
 {'_id': 'BETNetworks', 'AverageNumberOfLikes': 769687.625},
 {'_id': 'jypentertainment', 'AverageNumberOfLikes': 758826.6666666666},
 {'_id': 'Logan Paul Vlogs', 'AverageNumberOfLikes': 653443.5384615385},
 {'_id': 'Wengie', 'AverageNumberOfLikes': 567235.2},
 {'_id': 'Alesso', 'AverageNumberOfLikes': 509417.28571428574},
 {'_id': 'PewDiePie', 'AverageNumberOfLikes': 508647.8},
 {'_id': 'shakiraVEVO', 'AverageNumberOfLikes': 434302.14285714284},
 {'_id': 'Ed Sheeran', 'AverageNumberOfLikes': 395654.1666666667},
 {'_id': 'Dude Perfect', 'AverageNumberOfLikes': 389697.1111111111},
 {'_id': 'Conor Maynard', 'AverageNumberOfLikes': 389273.0},
 {'_id': 'TaylorSwiftVEVO', 'AverageNumberOfLikes': 367455.6666666667},
 {'_id': 'Liza Koshy', 'AverageNumberOfLikes': 365146.9583333333},
 {'_id': 'Linkin Park', 'AverageNum